# Decision Trees for Regression BNTX

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'BNTX'
start = '2020-01-01'
end = '2022-09-03'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,35.750000,38.500000,35.360001,38.500000,37.996727,139500
2020-01-03,38.000000,41.250000,36.419998,40.049999,39.526466,176000
2020-01-06,40.110001,45.000000,40.099998,44.580002,43.997250,333300
2020-01-07,47.400002,48.849998,41.180000,43.340000,42.773460,562100
2020-01-08,44.299999,46.330002,44.180000,45.099998,44.510452,193900


In [3]:
# Create more data
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,-1)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,-1)
dataset['Return'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Return
Date,,,,,,,,,,
2020-01-03,38.000000,41.250000,36.419998,40.049999,39.526466,176000,1,1,1,0.040260
2020-01-06,40.110001,45.000000,40.099998,44.580002,43.997250,333300,1,1,-1,0.113109
2020-01-07,47.400002,48.849998,41.180000,43.340000,42.773460,562100,0,-1,1,-0.027815
2020-01-08,44.299999,46.330002,44.180000,45.099998,44.510452,193900,1,1,-1,0.040609
2020-01-09,46.549999,46.709999,42.709999,43.270000,42.704376,234900,1,-1,-1,-0.040576


In [4]:
dataset.shape

(673, 10)

In [5]:
X = dataset.drop(['Adj Close', 'Close'], axis=1)  
y = dataset['Adj Close'] 

In [6]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

In [7]:
from sklearn.tree import DecisionTreeRegressor  
regressor = DecisionTreeRegressor()  
regressor.fit(X_train, y_train)

DecisionTreeRegressor()

In [8]:
from mlxtend.evaluate import bias_variance_decomp

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(regressor, X_train.values, y_train.values, X_test.values, y_test.values, loss='mse', random_seed=123)

print('MSE: %.3f' % avg_expected_loss)
print('Bias: %.3f' % avg_bias)
print('Variance: %.3f' % avg_var)

MSE: 59.215
Bias: 26.158
Variance: 33.057


In [9]:
y_pred = regressor.predict(X_test)

In [10]:
df = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})  
print(df.head())
print(df.tail())

                Actual   Predicted
Date                              
2022-03-09  138.377182  138.485748
2021-12-30  253.452972  241.965134
2021-11-02  285.695923  285.962372
2021-04-30  185.818802  190.250107
2020-01-24   34.532612   36.220257
                Actual   Predicted
Date                              
2020-04-24   47.125813   46.967903
2022-08-11  158.589996  161.029999
2020-09-30   68.325027   65.907051
2020-10-08   86.800316   89.198547
2021-10-22  274.701538  270.359070


In [11]:
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))  

Mean Absolute Error: 3.928795524879738
Mean Squared Error: 43.432345182353096
Root Mean Squared Error: 6.590322084872112


In [12]:
print(y_test.shape)
print(y_pred.shape)

(135,)
(135,)


In [13]:
from sklearn.model_selection import cross_val_score

dt_fit = regressor.fit(X_train, y_train)
dt_scores = cross_val_score(dt_fit, X_train, y_train, cv = 5)

print("Mean cross validation score: {}".format(np.mean(dt_scores)))
print("Score without cv: {}".format(dt_fit.score(X_train, y_train)))

Mean cross validation score: 0.9952992855519899
Score without cv: 1.0


In [14]:
from sklearn.metrics import r2_score

print('r2 score:', r2_score(y_test, dt_fit.predict(X_test)))
print('Accuracy Score:', dt_fit.score(X_test, y_test))

r2 score: 0.9959999390629526
Accuracy Score: 0.9959999390629526
